<a href="https://colab.research.google.com/github/MoseAIML/Moses-/blob/main/Copy_of_Imbalanced_Classification_Moses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problem Statement**
Beta Bank customers are leaving: little by little, chipping away every month. The bankers
figured out it’s cheaper to save the existing customers rather than to attract new ones.
We need to predict whether a customer will leave the bank soon. You have the data on
clients’ past behavior and termination of contracts with the bank.
Build a model with the maximum possible F1 score. To pass the project, you need an F1
score of at least 0.59. Check the F1 for the test set.
Additionally, measure the AUC-ROC metric and compare it with the F1.
1. Download and prepare the data. Explain the procedure.
2. Examine the balance of classes. Train the model without taking into account the
imbalance. Briefly describe your findings.
3. Improve the quality of the model. Make sure you use at least two approaches to
fixing class imbalance. Use the training set to pick the best parameters. Train
different models on training and validation sets. Find the best one. Briefly
describe your findings.
4. Perform the final testing.


In [ ]:
!pip install fsspec

     |████████████████████████████████| 134 kB 4.3 MB/s 


In [ ]:
#import libraries
import pandas as pd

#load dataset
df = pd.read_csv("https://bit.ly/2XZK7Bo")

# **Data preprocessing & analysis**

In [ ]:
df.sample(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
6139,6140,15806407,Wilson,652,France,Female,37,4.0,0.00,2,1,0,143393.24,0
5475,5476,15748608,Trentini,612,Germany,Male,42,5.0,141927.10,1,1,1,43018.98,0
4408,4409,15795963,Fiorentini,687,France,Male,34,7.0,129895.19,1,0,1,28698.17,0
3452,3453,15722965,Yefimova,757,France,Male,57,3.0,89079.41,1,1,1,53179.21,1
8203,8204,15588566,Wilkinson,778,Spain,Male,33,5.0,116474.28,2,1,1,32757.55,0


In [ ]:
#size of the dataset
df.shape

(10000, 14)

In [ ]:
#find missing values in the columns
df.isnull().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

In [ ]:
#fill missing values

#find unique values in Tenure column
df['Tenure'].unique()

#fill missing values with median
df['Tenure'] = df['Tenure'].fillna(df['Tenure'].median())

#check missing values
df['Tenure'].isnull().sum()

0

In [ ]:
#check for duplicates
df.duplicated().sum()

0

In [ ]:
#find datatypes in the columns
df.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure             float64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [ ]:
#drop some columns
df = df.drop(["Surname","RowNumber", "CustomerId"], axis=1)

In [ ]:
#encoding the string values in the dataset
df = pd.get_dummies(df, columns=['Gender', 'Geography'])

In [ ]:
df['Exited'].unique()

array([1, 0])

# **Model Training & Evaluation**

In [ ]:
#Splitting Dataset

#import libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

target = df['Exited']
features = df.drop('Exited', axis=1)
#split dataset
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345
)


#feature scaling
scaler = StandardScaler()
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)


### **Decision Tree Classifier**




In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=12345, max_depth = 2)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

print(f1_score(target_valid, predicted_valid))

0.5287846481876333


In [ ]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(random_state=12345, solver ='liblinear')
lr_model.fit(features_train, target_train)
predicted_valid = lr_model.predict(features_valid)

print(f1_score(target_valid, predicted_valid))

0.29247910863509746


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=12345, n_estimators=50)
rf_model.fit(features_train, target_train)
predicted_valid = rf_model.predict(features_valid)

print(f1_score(target_valid, predicted_valid))


0.5810968494749125


# **Class Balance Adjustments**

In [ ]:
#check balance of target class
df['Exited'].value_counts(normalize=True)

0    0.7963
1    0.2037
Name: Exited, dtype: float64

In [ ]:
#balanced random forest model
rf_model_balanced = RandomForestClassifier(random_state=12345, n_estimators=50, class_weight='balanced')
rf_model_balanced.fit(features_train, target_train)
predicted_valid = rf_model_balanced.predict(features_valid)

print(f1_score(target_valid, predicted_valid))


0.5621230398069964


In [ ]:
#balanced Decision Tree
balanced_model = DecisionTreeClassifier(random_state=12345, max_depth = 2, class_weight='balanced')
balanced_model.fit(features_train, target_train)
predicted_valid = balanced_model.predict(features_valid)

print(f1_score(target_valid, predicted_valid))

0.5391566265060241
